In [6]:
import numpy as np
import pandas as pd
import redpitaya_scpi as scpi
import time
import datetime
import os
import json
import joblib
from scipy.signal import welch
from sklearn.preprocessing import StandardScaler


# Function to load configurations from config.json
def load_config(config_path='config.json'):
    if not os.path.exists(config_path):
        raise FileNotFoundError(f"Configuration file '{config_path}' not found.")
    
    with open(config_path, 'r') as config_file:
        config = json.load(config_file)
    
    required_keys = ["ip_address", "decimation_factor", "scaler_path", "model_path"]
    for key in required_keys:
        if key not in config:
            raise KeyError(f"Missing required configuration key: '{key}'")
    
    return config


# Function to compute statistical features
def compute_statistical_features(signal):
    """
    Compute statistical features from the Welch power spectrum of a signal.
    """
    try:
        features = {
            'entropy': np.sum(-signal * np.log2(signal + 1e-12)),  # Avoid log(0)
            'skewness': pd.Series(signal).skew(),
            'interquartile_range': np.percentile(signal, 75) - np.percentile(signal, 25),
            'kurtosis': pd.Series(signal).kurtosis(),
            'percentile_75': np.percentile(signal, 75),
            'range': np.ptp(signal),
            'maximum': np.max(signal),
            'median': np.median(signal),
            'percentile_90': np.percentile(signal, 90),
            'mean_absolute_deviation': np.mean(np.abs(signal - np.mean(signal)))
        }
    except Exception as e:
        print(f"Error computing features: {e}")
        features = {key: np.nan for key in [
            'entropy', 'skewness', 'interquartile_range', 'kurtosis', 'percentile_75',
            'range', 'maximum', 'median', 'percentile_90', 'mean_absolute_deviation']}
    
    return features


# Function to process live data and classify using the KNN model
def classify_live_data(signal, scaler, model, decimation_factor):
    """
    Process live data from Red Pitaya and classify it using the trained model.
    """
    # Compute Welch power spectrum
    frequencies, power_spectrum = welch(signal, fs=125e6 / decimation_factor, nperseg=1024)
    power_spectrum = power_spectrum / np.sum(power_spectrum)  # Normalize to get probabilities
    
    # Compute statistical features
    features = compute_statistical_features(power_spectrum)
    feature_array = np.array(list(features.values())).reshape(1, -1)
    
    # Scale features and classify
    scaled_features = scaler.transform(feature_array)
    probabilities = model.predict_proba(scaled_features)[0]
    
    # Map probabilities to device labels
    device_labels = model.classes_
    timestamp = datetime.datetime.now().isoformat()
    classification_result = {'timestamp': timestamp}
    classification_result.update({label: prob for label, prob in zip(device_labels, probabilities)})
    
    return classification_result


# Main data collection and classification function
def collect_and_classify(config):
    # Extract configurations
    ip_address = config['ip_address']
    decimation_factor = config['decimation_factor']
    scaler_path = config['scaler_path']
    model_path = config['model_path']
    
    # Load scaler and model
    scaler = joblib.load(scaler_path)
    model = joblib.load(model_path)
    
    # Initialize Red Pitaya connection
    try:
        rp_s = scpi.scpi(ip_address)
        print(f"Connected to Red Pitaya at {ip_address}")
    except Exception as e:
        print(f"Failed to connect to Red Pitaya at {ip_address}: {e}")
        return
    
    try:
        while True:
            # Configure Red Pitaya
            try:
                rp_s.tx_txt('ACQ:RST')  # Reset acquisition
                rp_s.tx_txt(f'ACQ:DEC {decimation_factor}')  # Set decimation factor
                rp_s.tx_txt('ACQ:START')  # Start acquisition
                time.sleep(1)  # Wait for data capture
            except Exception as e:
                print(f"Error configuring Red Pitaya: {e}")
                time.sleep(2)
                continue  # Skip to next iteration
            
            # Capture data from channel 1
            try:
                rp_s.tx_txt('ACQ:SOUR1:DATA?')
                raw_data_ch1 = rp_s.rx_txt().strip('{}\n\r').split(',')
                signal_ch1 = np.array([float(x) for x in raw_data_ch1])
                signal_ch1 -= np.mean(signal_ch1)  # Remove DC component
            except Exception as e:
                print(f"Error acquiring data from channel 1: {e}")
                time.sleep(1)
                continue  # Skip to next iteration
            
            # Classify live data
            if signal_ch1.size > 0:
                try:
                    result = classify_live_data(signal_ch1, scaler, model, decimation_factor)
                    print(f"Classification result: {result}")
                except Exception as e:
                    print(f"Error classifying live data: {e}")
            else:
                print("No valid data captured from channel 1.")
            
            # Optional: control loop frequency
            time.sleep(2)  # Adjust the delay as needed
    except KeyboardInterrupt:
        print("\nStopped by user.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")


# Entry point
if __name__ == "__main__":
    try:
        config = load_config('config.json')
    except (FileNotFoundError, KeyError) as e:
        print(f"Configuration Error: {e}")
        sys.exit(1)
    
    collect_and_classify(config)


Connected to Red Pitaya at 192.168.8.214
Classification result: {'timestamp': '2024-12-02T15:05:24.190205', 'empty': 1.0, 'ipad': 0.0, 'lamp': 0.0}
Classification result: {'timestamp': '2024-12-02T15:05:27.705742', 'empty': 1.0, 'ipad': 0.0, 'lamp': 0.0}

Stopped by user.
